### Implementing Data Validation using Protobuf in a Distributed System
**Description**: Use gRPC to implement a distributed system that validates messages using
Protobuf.

**Steps**:
1. Create a .proto file for gRPC service.
2. Implement server-side validation
    - Create a gRPC server
    - Bind the server to an address
    - Start server

In [ ]:
import os
import sys
import re
import grpc
import tempfile
from concurrent import futures

# Write .proto content to a file
proto_content = '''
syntax = "proto3";
package user;

service UserValidator {
  rpc Validate (UserRequest) returns (ValidationResult);
}

message UserRequest {
  int32 id = 1;
  string name = 2;
  int32 age = 3;
  string email = 4;
}

message ValidationResult {
  bool is_valid = 1;
  string message = 2;
}
'''

# Create a temp dir to store the proto and generated files
tmpdir = tempfile.mkdtemp()
proto_path = os.path.join(tmpdir, "user.proto")

with open(proto_path, "w") as f:
    f.write(proto_content)

# Compile proto using grpc_tools
from grpc_tools import protoc

protoc.main((
    '',
    f'-I{tmpdir}',
    f'--python_out={tmpdir}',
    f'--grpc_python_out={tmpdir}',
    proto_path,
))

# Add tmpdir to sys.path to import generated modules
sys.path.insert(0, tmpdir)
import user_pb2, user_pb2_grpc

# Implement the server logic
class UserValidatorService(user_pb2_grpc.UserValidatorServicer):
    def Validate(self, request, context):
        if not request.name or not request.email:
            return user_pb2.ValidationResult(is_valid=False, message="Name and email are required.")
        if request.age < 18 or request.age > 100:
            return user_pb2.ValidationResult(is_valid=False, message="Age must be between 18 and 100.")
        if not re.match(r"[^@]+@[^@]+\.[^@]+", request.email):
            return user_pb2.ValidationResult(is_valid=False, message="Invalid email format.")
        return user_pb2.ValidationResult(is_valid=True, message="User is valid.")

# Start gRPC server
def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=5))
    user_pb2_grpc.add_UserValidatorServicer_to_server(UserValidatorService(), server)
    server.add_insecure_port('[::]:50051')
    print("Server started on port 50051")
    server.start()
    server.wait_for_termination()

if __name__ == "__main__":
    serve()
